In [1]:
'modified by abdul alkurdi; 10/05/2023'

import pandas as pd
import dask.dataframe as dd
import dask
#import cudf
import pickle
import json
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import figure
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.signal import correlate

import pyarrow.parquet as pq
import os
from datetime import datetime
#import heartpy as hp
import json
%matplotlib notebook
# import neurokit2 as nk

import process_redcap 
from syncfcns import *

AttributeError: module 'pandas.arrays' has no attribute 'ArrowStringArray'

In [ ]:
radwear_path = '/mnt/c/Users/alkurdi/Desktop/Vansh/data/RADWear/'
redcap_path = radwear_path+'REDCap responses/'

# load all participant meta data
with open(radwear_path+'all_p_metadata.json', 'rb') as f:
            all_p_metadata = json.load(f)
# load all participant redcap data
redcap_df = process_redcap.process_redcap(redcap_path,all_p_metadata['list of participant IDs'])
list_of_participants = all_p_metadata['list of participant IDs']
completed_participants = []



filepath = '/mnt/c/Users/alkurdi/Desktop/Vansh/data/RADWear/'
# label definitions
calib_dict = {'meditation': 0, 'cpt': 1}
rot_anx_dict = {'calibration': 0, 'LA': 1, 'HA': 2}

p_calib = {}
p_la = {}
p_ha = {}
p_all = {} 
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]

In [ ]:
def read_sync_return(filepath, e4_today, hx_today):
    '''
    usage: read_sync_return(filepath, e4_today, hx_today)
    returns: synced_participant_data
    '''
    e4_dict = read_E4(filepath+'/'+e4_today, hx_today)
    hx_dict = read_hx(filepath+'/record_'+hx_today, hx_today)

    #e4sync_offset = E4sync_offset(e4_dict)
    #hxsync_offset = Hexsync_offset(hx_dict['ECG'],hx_dict['BR'],hx_dict['accx'],hx_dict['accy'], hx_dict['accz'])
    e4sync_offset = e4_dict
    hxsync_offset = hx_dict
    
    synced_participant_data = {}
    #cross syncing between devices but only on bvp and ecg
    synced_participant_data['ECG'], synced_participant_data['BVP'] = doublesync_offset(hxsync_offset['ECG'], e4sync_offset['BVP'])
    # for some reason we're losing seconds from beginning and end of hx data even though e4 is contained within it's timeframe.


    # running syncing on rest of signals for hx (ECG, breathing_rateacc, br)
    t0 = synced_participant_data['ECG']['Second'].iloc[0] #hx
    tf = synced_participant_data['ECG']['Second'].iloc[-1] #hx
    synced_participant_data['BR'] = hxsync_offset['BR'].loc[hxsync_offset['BR']['Second'] >= t0 ].loc[hxsync_offset['BR']['Second'] <= tf ]
    del tf, t0

    # running syncing on rest of signals for e4 (acc, eda, temp, hr, ibi )
    t0 = synced_participant_data['BVP']['Second'].iloc[0] #e4
    tf = synced_participant_data['BVP']['Second'].iloc[-1] #e4
    synced_participant_data['TEMP'] = e4sync_offset['TEMP'].loc[e4sync_offset['TEMP']['Second'] >= t0 ].loc[e4sync_offset['TEMP']['Second'] <= tf ]
    synced_participant_data['EDA'] = e4sync_offset['EDA'].loc[e4sync_offset['EDA']['Second'] >= t0 ].loc[e4sync_offset['EDA']['Second'] <= tf ]
    synced_participant_data['HR'] = e4_dict['HR'].loc[e4_dict['HR']['Second'] >= t0 ].loc[e4_dict['HR']['Second'] <= tf ]
    #synced_participant_data['IBI'] = e4sync_offset['IBI'].loc[e4sync_offset['IBI']['Second'] >= t0 ].loc[e4sync_offset['IBI']['Second'] <= tf ]
    del tf, t0

    hx_dict['ACC'] = hx_dict['accx']
    hx_dict['ACC']['Acc_Y'] = hx_dict['accy']['Acc_Y']
    hx_dict['ACC']['Acc_Z'] = hx_dict['accz']['Acc_Z']
     
    acc_dic = accsync_offset(e4sync_offset['ACC'],hxsync_offset['accx'])
    #offset_ecg_cross, offset_bvp_cross = doublesync_offset(e4sync_offset['ECG'], e4sync_offset['BVP'])
    synced_participant_data['ACC_hx'] = acc_dic['acc_hx']
    synced_participant_data['ACC_e4'] = acc_dic['acc_e4']

    return synced_participant_data
    


In [ ]:
#########################
# p5 HA day 8 and p9 HA day 10 will be done manually. redcap is marked as 0 but data is available
#########################

# wesad.keys() = dict_keys(['signal', 'label', 'subject']); label returns array subject: S$ 
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]

for p in list_of_participants:
    # check if participant pickle file exists
    if Path(radwear_path+'Participant '+str(p)+'/p_'+str(p)+'.pkl').is_file():
        print(radwear_path+'Participant '+str(p)+'/p_'+str(p)+'.pkl')
        print('participant ', p, ' already processed.')
        completed_participants.append(p)
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]


In [ ]:
print(completed_participants)
print(incomplete)
p_all = {}

In [ ]:
for p in incomplete:
    p_la = {}
    p_ha = {}
    p_all = {}
    print('procesing participant: ', p, ' calibration data...')
    p_all[p] = {}
    p_la[p] = {}
    p_ha[p] = {}
    p_calib = {}

    p_path = radwear_path+'Participant '+str(p)
    # load participant e4 data
    e4sn = all_p_metadata[str(p)]['e4sn']
    calibration_files = all_p_metadata[str(p)]['calibration']
    LA = all_p_metadata[str(p)]['LA']
    HA = all_p_metadata[str(p)]['HA']
    
    # load calibration data
    e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['calibration'][0]
    hx_num = str(all_p_metadata[str(p)]['calibration'][1])
    p_calib[p] = read_sync_return(p_path, e4_num, hx_num)
    p_calib[p]['rot_label'] = rot_anx_dict['calibration'] * np.ones(len(p_calib[p]['ECG'])) # add label to designate calibration segment
    p_calib[p] = dd.from_dict(p_calib[p]) # for dask conversion
    
    #####################
    # for when adding labels for cpt and meditation
    #calib_label = for 
    
    if all_p_metadata[str(p)]['complete days'][0] > 0:
        # loop for LA
        for i in range(all_p_metadata[str(p)]['complete days'][0]): # loop for LA
            if not (all_p_metadata[str(p)]['RedCap available'][0][i] == 0 or all_p_metadata[str(p)]['LA'][0][i] == 0 or all_p_metadata[str(p)]['LA'][1][i] == 0): 
                print('procesing participant: ', p, ' LA day ', i+1, '...')    
                e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['LA'][0][i]
                hx_num = str(all_p_metadata[str(p)]['LA'][1][i])
                
                p_la[p][i] = read_sync_return(p_path, e4_num, hx_num)
                p_la[p][i]['rot_label']= rot_anx_dict['LA'] * np.ones(len(p_la[p][i]['ECG'])) 
            else:
                p_la[p][i] = {}
                p_la[p][i] = dd.from_dict(p_la[p][i]) # for dask conversion 
    if all_p_metadata[str(p)]['complete days'][1] > 0:
        # loop for HA    
        for j in range(all_p_metadata[str(p)]['complete days'][1]): # loop for HA
            if not (all_p_metadata[str(p)]['RedCap available'][1][j] == 0 or all_p_metadata[str(p)]['HA'][0][j] == 0 or all_p_metadata[str(p)]['HA'][1][j] == 0): 
                print('procesing participant: ', p, ' HA day ', j+1, '...')    
                e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['HA'][0][j]
                hx_num = str(all_p_metadata[str(p)]['HA'][1][j])
                
                p_ha[p][j] = read_sync_return(p_path, e4_num, hx_num)
                p_ha[p][j]['rot_label']= rot_anx_dict['HA'] * np.ones(len(p_ha[p][j]['ECG']))
            else:
                p_ha[p][j] = {}
                p_ha[p][j] = dd.from_dict(p_ha[p][j]) # for dask conversion
    print('participant ', p, ' done.')
    if True:
        try:
            p_all[p]['calib']= p_calib[p]
        except:
            print('participant ', p, ' has no calibration data.')
            p_all[p]['calib'] = {}
        try:    
            p_all[p]['LA'] = p_la[p]
        except:
            print('participant ', p, ' has no LA data.')
            p_all[p]['LA'] = {}      
        try: 
            p_all[p]['HA'] = p_ha[p]
        except:
            print('participant ', p, ' has no HA data.')
            p_all[p]['HA'] = {}
        #save this participant data to a pickle file
        #with open(p_path+'/p_'+str(p)+'.pkl', 'wb') as handle:
        #        pickle.dump(p_all[p], handle, protocol=pickle.HIGHEST_PROTOCOL)
        #pq.write_table(p_all[p], p_path+'/p_'+str(p)+'.pkl')
        # or 
        df.to_parquet(p_path+'/p_'+str(p)+'_parq')
        print('participant ', p, ' saved.')
    completed_participants.append(p)
    print('-----------------------------------')

print('completed participants: ', completed_participants)
print('incomplete participants: ', incomplete)

In [ ]:
p_la[p][i] = read_sync_return(p_path, e4_num, hx_num)

In [ ]:
x = 17
e4_num = all_p_metadata[str(x)]['e4sn']+'_'+'230817-203116'
hx_num = 'record_271784'
p_path = radwear_path+'Participant '+str(x)
print(e4_num)
print(hx_num)
print(p_path)
print(p_path+'/'+e4_num)
print(p_path+'/'+hx_num)
e4_dict = read_E4(p_path+'/'+e4_num, hx_num)
hx_dict = read_hx(p_path+'/'+hx_num, hx_num)


In [ ]:
print(e4_dict.keys())
print(hx_dict.keys())

In [ ]:
hx_dict['ECG']


In [ ]:
hx_dict['ECG']
print('total time is = ', 32572/60/60,'hours')

In [ ]:
p_path = radwear_path+'Participant '+str(9)
p_path

with open(p_path+'/p_'+str(9)+'.pkl', 'rb') as handle:
    p9 = pickle.load(handle)

In [ ]:
print(p_la.keys())
print(p_ha.keys())

In [ ]:
wesad_path = '/mnt/c/Users/alkurdi/Desktop/Vansh/data/WESAD/'
with open(wesad_path+'S2/S2.pkl', 'rb') as f:
            wesad = pickle.load(f,encoding='latin1')



In [ ]:
for i in wesad['signal']['wrist'].keys():
    print(i)
print('\n\n')
for i in wesad['signal']['wrist'].keys():
    print(len(wesad['signal']['wrist'][i]))
    
for i in wesad['signal']['chest'].keys():
    print(i)
print('\n\n')
for i in wesad['signal']['chest'].keys():
    print(len(wesad['signal']['chest'][i]))

In [ ]:
for i in hx_dict.keys():
    print(i)
print('\n\n')
for i in hx_dict.keys():
    print(len(hx_dict[i]))
for i in e4_dict.keys():
    print(i)
print('\n\n')
for i in e4_dict.keys():
    print(len(e4_dict[i]))

# TAGS